# Training a deep (convolutional) neural network on CIFAR10

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
import os

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import time
import PIL.Image as Image
from IPython.display import Image, display
import matplotlib.cm as cm

print(tf.__version__)
print(tf.config.list_physical_devices())



## 1. Code Convolutions

In [ ]:
img =  tf.keras.utils.load_img('yourImage.jpeg', target_size=(128,128))
data =  tf.keras.utils.img_to_array(img)/255.0
samples = tf.expand_dims(data, 0)


plt.imshow(img, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
img =  tf.keras.utils.load_img('yourImage.jpeg', target_size=(128,128))
data =  tf.keras.utils.img_to_array(img)/255.0
samples = tf.expand_dims(data, 0)

channels = 3


# Create a 7x7 vertical filter and then a horizontal filter
# ATTENTION: Convolutions in keras are 4D, meaning [num_pixels, num_pixels, num_channels, num_filters]

#define the structure of the filter
filter = #ADD YOUR CODE

#assign the right numerical values to the array (filters weights)
filter[#ADD YOUR CODE
    ] = #ADD YOUR CODE



outputs = tf.nn.conv2d(samples, filter, strides=1, padding="SAME")



plt.imshow(outputs[0, :, :, 0], cmap="gray") # plot image's 1nd feature map
plt.axis("off")
plt.show()

## Ex2: CNN on CIFAR 10

In [ ]:
#setup folder where you will save logs for tensorflow
root_logdir = os.path.join(os.curdir,"my_logs_ML2_CIFAR")


def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)



Import the CIFAR 10 library, split into train, validation and test images.

In [ ]:
cifar = tf.keras.datasets.cifar10

(train_full_images, train_full_labels), (test_images, test_labels) = cifar.load_data()

valid_images, train_images = train_full_images[:5000], train_full_images[5000:]
valid_labels, train_labels = train_full_labels[:5000], train_full_labels[5000:]
test_images = test_images

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays,
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

Check out some properties of the imported dataset

In [ ]:
print('training ds shape:', train_images.shape)
print('validation ds shape:', valid_images.shape)
print('test ds shape:', test_images.shape)

In [ ]:
print("labels: ", train_labels)

In [ ]:
plt.figure()
plt.imshow(train_images[1])
plt.colorbar()
plt.grid(False)
plt.show()

Normalization of data between 0 and 1

In [ ]:
train_images = train_images / 255.0

valid_images = valid_images / 255.0

test_images = test_images / 255.0

In [ ]:
run_logdir = get_run_logdir()

#create keras Tensorboard callback and early stopping_callback
tensorboard_cb = #ADD YOUR CODE
earlystopping_cb  = #ADD YOUR CODE

In [ ]:
model = tf.keras.Sequential([
    #ADD YOUR CODE
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')
])


In [ ]:
model.summary()

In [ ]:
model.compile( #ADD YOUR CODE
    )

In [ ]:
model.fit( #ADD YOUR CODE
    )

In [ ]:
%load_ext tensorboard
%tensorboard --logdir my_logs_ML2_CIFAR

### Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[int(true_label)]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[int(true_label)].set_color('blue')

In [ ]:
predictions = model.predict(test_images)

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

## Use the trained model

Finally, use the trained model to make a prediction about a single image.

In [ ]:
img = tf.keras.utils.load_img("./test_image.jpg", target_size=(32,32)#download your own image
)

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Add the image to a batch where it's the only member.


predictions = model.predict(img_array)
score = predictions[0]

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)